<a href="https://colab.research.google.com/github/Shaitender/Projects-poc/blob/master/RepeatBuyersPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!unzip /content/data_format1.zip -d /content/data_format1
#!unzip /content/data_format2.zip -d /content/data_format2

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [0]:
data_format1_paths = '/content/data_format1'
data_format2_paths = '/content/data_format2'

# data_format 1
data = pd.read_csv(f'{data_format1_paths}/user_log_format1.csv', dtype={'time_stamp':'str'})
data1 = pd.read_csv(f'{data_format1_paths}/user_info_format1.csv')
data2 = pd.read_csv(f'{data_format1_paths}/train_format1.csv')
submission = pd.read_csv(f'{data_format1_paths}/test_format1.csv')

# data_format 2
data_train = pd.read_csv(f'{data_format2_paths}/test_format2.csv')
data_test = pd.read_csv(f'{data_format2_paths}/test_format2.csv')

In [4]:
data.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,0829,0
1,328862,844400,1271,2882,2661.0,0829,0
2,328862,575153,1271,2882,2661.0,0829,0
3,328862,996875,1271,2882,2661.0,0829,0
4,328862,1086186,1271,1253,1049.0,0829,0


In [5]:
data1.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [6]:
data2.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [7]:
submission.head()

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN


In [8]:
data_train.head()

,user_id,age_range,gender,merchant_id,label,activity_log
0,163968,0.0,0.0,4378,-1.0,101206:812:6968:0614:0
1,163968,0.0,0.0,2300,-1.0,588758:844:3833:0618:0#71782:844:3833:1111:2#7...
2,163968,0.0,0.0,1551,-1.0,312747:243:1954:0627:0#312747:243:1954:0627:0#...
3,163968,0.0,0.0,4343,-1.0,932390:1612:3201:0628:0
4,163968,0.0,0.0,4911,-1.0,957657:662:3089:0612:0


In [9]:
data_test.head()

,user_id,age_range,gender,merchant_id,label,activity_log
0,163968,0.0,0.0,4378,-1.0,101206:812:6968:0614:0
1,163968,0.0,0.0,2300,-1.0,588758:844:3833:0618:0#71782:844:3833:1111:2#7...
2,163968,0.0,0.0,1551,-1.0,312747:243:1954:0627:0#312747:243:1954:0627:0#...
3,163968,0.0,0.0,4343,-1.0,932390:1612:3201:0628:0
4,163968,0.0,0.0,4911,-1.0,957657:662:3089:0612:0


In [0]:
data2['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([data2, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(data1, on='user_id', how='left')
data.rename(columns={'seller_id':'merchant_id'}, inplace=True)

In [11]:
data['user_id'] = data['user_id'].astype('int32')
data['merchant_id'] = data['merchant_id'].astype('int32')
data['item_id'] = data['item_id'].astype('int32')
data['cat_id'] = data['cat_id'].astype('int32')
data['brand_id'].fillna(0, inplace=True)
data['brand_id'] = data['brand_id'].astype('int32')
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%H%M')

matrix['age_range'].fillna(0, inplace=True)
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

import gc

del data1, data2
gc.collect()

4

In [0]:
groups = data.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

In [0]:
groups = data.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'user_id':'m2',
    'item_id':'m3', 
    'cat_id':'m4', 
    'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [0]:
matrix.head(100)

In [0]:
temp = data_train[data_train['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [0]:
matrix.head()

In [0]:
groups = data.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'item_id':'um2',
    'cat_id':'um3',
    'brand_id':'um4'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={
    0:'um5',
    1:'um6',
    2:'um7',
    3:'um8'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('frist', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['frist']).dt.seconds/3600
temp.drop(['frist', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
